In [2]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [10]:
# Create drive to access Google Drive
gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
gauth.LoadClientConfigFile("credentials.json")
drive = GoogleDrive(gauth)

In [11]:
download = drive.CreateFile({'id': '1BZOv422XJvxFUnGh-0xVeSvgFgqVY45q'})

In [12]:
download.GetContentFile('train_LbELtWX.zip')


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=772264689031-rqes5qk1co2il7mnl5cu00h0vrnu6d4h.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [13]:
!unzip train_LbELtWX.zip

Archive:  train_LbELtWX.zip
   creating: train/
 extracting: train/3751.png          
 extracting: train/1986.png          
 extracting: train/46774.png         
 extracting: train/1386.png          
 extracting: train/11162.png         
 extracting: train/52576.png         
 extracting: train/24590.png         
 extracting: train/780.png           
 extracting: train/46235.png         
 extracting: train/22625.png         
 extracting: train/48961.png         
 extracting: train/51857.png         
 extracting: train/15658.png         
 extracting: train/6660.png          
 extracting: train/46576.png         
 extracting: train/23201.png         
 extracting: train/26622.png         
 extracting: train/16475.png         
 extracting: train/50593.png         
 extracting: train/8611.png          
 extracting: train/2259.png          
 extracting: train/23122.png         
 extracting: train/20565.png         
 extracting: train/59996.png         
 extracting: train/16779.png         
 e

 extracting: train/35385.png         
 extracting: train/38153.png         
 extracting: train/30900.png         
 extracting: train/5022.png          
 extracting: train/31570.png         
 extracting: train/41426.png         
 extracting: train/36148.png         
 extracting: train/32854.png         
 extracting: train/51435.png         
 extracting: train/10838.png         
 extracting: train/21830.png         
 extracting: train/20960.png         
 extracting: train/58358.png         
 extracting: train/11816.png         
 extracting: train/56891.png         
 extracting: train/48985.png         
 extracting: train/45311.png         
 extracting: train/32775.png         
 extracting: train/52985.png         
 extracting: train/25051.png         
 extracting: train/18020.png         
 extracting: train/27401.png         
 extracting: train/43181.png         
 extracting: train/56692.png         
 extracting: train/26612.png         
 extracting: train/57309.png         
 extracting:

 extracting: train/28142.png         
 extracting: train/8359.png          
 extracting: train/33169.png         
 extracting: train/30142.png         
 extracting: train/10730.png         
 extracting: train/1960.png          
 extracting: train/11147.png         
 extracting: train/29268.png         
 extracting: train/59448.png         
 extracting: train/48018.png         
 extracting: train/36784.png         
 extracting: train/33507.png         
 extracting: train/49057.png         
 extracting: train/18665.png         
 extracting: train/47387.png         
 extracting: train/51718.png         
 extracting: train/50775.png         
 extracting: train/34540.png         
 extracting: train/10455.png         
 extracting: train/5718.png          
 extracting: train/52070.png         
 extracting: train/15255.png         
 extracting: train/14738.png         
 extracting: train/9385.png          
 extracting: train/48015.png         
 extracting: train/40990.png         
 extracting:

 extracting: train/46285.png         
 extracting: train/19762.png         
 extracting: train/36774.png         
 extracting: train/53999.png         
 extracting: train/35334.png         
 extracting: train/29717.png         
 extracting: train/29479.png         
 extracting: train/29908.png         
 extracting: train/50471.png         
 extracting: train/34521.png         
 extracting: train/55545.png         
  inflating: train/25896.png         
 extracting: train/51879.png         
 extracting: train/31454.png         
 extracting: train/39095.png         
 extracting: train/40209.png         
  inflating: train/51611.png         
 extracting: train/11151.png         
 extracting: train/6340.png          
 extracting: train/34617.png         
 extracting: train/4555.png          
 extracting: train/7910.png          
 extracting: train/14217.png         
 extracting: train/13681.png         
 extracting: train/39857.png         
 extracting: train/4932.png          
 extracting:

 extracting: train/4996.png          
 extracting: train/42048.png         
 extracting: train/59519.png         
 extracting: train/32473.png         
 extracting: train/25080.png         
 extracting: train/19639.png         
 extracting: train/47119.png         
 extracting: train/23008.png         
 extracting: train/40885.png         
 extracting: train/17347.png         
 extracting: train/31884.png         
 extracting: train/41131.png         
 extracting: train/28929.png         
 extracting: train/19033.png         
 extracting: train/45435.png         
 extracting: train/51972.png         
 extracting: train/56846.png         
 extracting: train/11469.png         
 extracting: train/13543.png         
 extracting: train/22632.png         
 extracting: train/30062.png         
 extracting: train/47273.png         
 extracting: train/25076.png         
 extracting: train/47096.png         
 extracting: train/5970.png          
 extracting: train/4603.png          
 extracting:

...

 extracting: train/39534.png         
 extracting: train/31686.png         
 extracting: train/16745.png         
 extracting: train/23621.png         
 extracting: train/15628.png         
 extracting: train/9292.png          
 extracting: train/20249.png         
 extracting: train/32053.png         
 extracting: train/46794.png         
 extracting: train/20829.png         
 extracting: train/42510.png         
 extracting: train/48082.png         
 extracting: train/5646.png          
 extracting: train/23855.png         
 extracting: train/12640.png         
 extracting: train/12180.png         
 extracting: train/19343.png         
 extracting: train/3777.png          
 extracting: train/50938.png         
 extracting: train/4437.png          
 extracting: train/36964.png         
 extracting: train/59427.png         
 extracting: train/2876.png          
 extracting: train/36481.png         
 extracting: train/56062.png         
 extracting: train/11211.png         
 extracting:

 extracting: train/7104.png          
 extracting: train/47426.png         
 extracting: train/57317.png         
 extracting: train/5359.png          
 extracting: train/55921.png         
 extracting: train/20624.png         
 extracting: train/36595.png         
 extracting: train/46117.png         
 extracting: train/14900.png         
 extracting: train/26070.png         
 extracting: train/47996.png         
 extracting: train/19186.png         
 extracting: train/53351.png         
 extracting: train/43312.png         
 extracting: train/58448.png         
 extracting: train/7373.png          
 extracting: train/30406.png         
 extracting: train/29784.png         
 extracting: train/21795.png         
 extracting: train/3453.png          
 extracting: train/23673.png         
 extracting: train/15345.png         
 extracting: train/43426.png         
 extracting: train/9592.png          
 extracting: train/14311.png         
 extracting: train/37741.png         
 extracting:

 extracting: train/17874.png         
 extracting: train/39479.png         
 extracting: train/38804.png         
 extracting: train/56777.png         
 extracting: train/44798.png         
 extracting: train/56129.png         
 extracting: train/16688.png         
 extracting: train/12768.png         
 extracting: train/32689.png         
 extracting: train/6484.png          
 extracting: train/34177.png         
 extracting: train/31287.png         
 extracting: train/3617.png          
 extracting: train/42999.png         
 extracting: train/13933.png         
 extracting: train/55002.png         
 extracting: train/5334.png          
 extracting: train/38922.png         
 extracting: train/12800.png         
 extracting: train/20567.png         
 extracting: train/34737.png         
 extracting: train/48222.png         
 extracting: train/16861.png         
 extracting: train/31405.png         
 extracting: train/7427.png          
 extracting: train/33356.png         
 extracting:

 extracting: train/43409.png         
 extracting: train/30055.png         
 extracting: train/55876.png         
 extracting: train/53268.png         
 extracting: train/1718.png          
 extracting: train/4596.png          
 extracting: train/48302.png         
 extracting: train/8037.png          
 extracting: train/22806.png         
 extracting: train/48430.png         
 extracting: train/1787.png          
 extracting: train/27364.png         
 extracting: train/47515.png         
 extracting: train/36580.png         
 extracting: train/44433.png         
 extracting: train/32931.png         
 extracting: train/12793.png         
 extracting: train/17178.png         
 extracting: train/57278.png         
 extracting: train/47501.png         
 extracting: train/58112.png         
 extracting: train/44009.png         
 extracting: train/46824.png         
 extracting: train/46344.png         
 extracting: train/59449.png         
 extracting: train/6037.png          
 extracting:

 extracting: train/41398.png         
 extracting: train/37216.png         
 extracting: train/31205.png         
 extracting: train/23365.png         
 extracting: train/18976.png         
 extracting: train/11006.png         
 extracting: train/6633.png          
 extracting: train/21169.png         
 extracting: train/59281.png         
 extracting: train/24556.png         
 extracting: train/11659.png         
 extracting: train/36121.png         
 extracting: train/44099.png         
 extracting: train/30587.png         
 extracting: train/49042.png         
 extracting: train/22513.png         
 extracting: train/25813.png         
 extracting: train/54067.png         
 extracting: train/32504.png         
 extracting: train/52142.png         
 extracting: train/17234.png         
 extracting: train/19510.png         
 extracting: train/45929.png         
 extracting: train/13219.png         
 extracting: train/50974.png         
 extracting: train/45537.png         
 extracting:

In [34]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from PIL import Image

In [76]:
train = pd.read_csv('train.csv')

,id,label
0,1,9
1,2,0
2,3,0
3,4,3
4,5,0


In [37]:
train_images = []
for i in tqdm(range(train.shape[0])):
    img = Image.open('train/'+train['id'][i].astype('str')+'.png')
    img = image.img_to_array(img)
    img = img/255
    train_images.append(img)
X = np.array(train_images)

100%|██████████| 60000/60000 [00:20<00:00, 2958.00it/s]


In [43]:
len(y)

60000

In [86]:
len(X[0]) # Len 28
len(X[0][0]) # Len 28
len(X[0][0][0]) # Len 4

array([0., 0., 0., 1.], dtype=float32)

In [44]:
y = train['label'].values
y = to_categorical(y)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,random_state=42, test_size=0.2)

In [60]:
len(X_test)

12000

In [71]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,4)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [72]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [111]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 80s 2ms/step - loss: 0.1699 - accuracy: 0.9365 - val_loss: 0.2213 - val_accuracy: 0.9296
Epoch 2/10
48000/48000 [==============================] - 82s 2ms/step - loss: 0.1700 - accuracy: 0.9357 - val_loss: 0.2250 - val_accuracy: 0.9258
Epoch 3/10
48000/48000 [==============================] - 82s 2ms/step - loss: 0.1582 - accuracy: 0.9406 - val_loss: 0.2216 - val_accuracy: 0.9261
Epoch 4/10
48000/48000 [==============================] - 79s 2ms/step - loss: 0.1509 - accuracy: 0.9431 - val_loss: 0.2439 - val_accuracy: 0.9242
Epoch 5/10
48000/48000 [==============================] - 83s 2ms/step - loss: 0.1493 - accuracy: 0.9433 - val_loss: 0.2223 - val_accuracy: 0.9271
Epoch 6/10
48000/48000 [==============================] - 85s 2ms/step - loss: 0.1410 - accuracy: 0.9472 - val_loss: 0.2341 - val_accuracy: 0.9225
Epoch 7/10
48000/48000 [==============================] - 86s 2ms/st

In [88]:
# download = drive.CreateFile({'id': '1KuyWGFEpj7Fr2DgBsW8qsWvjqEzfoJBY'})
# download.GetContentFile('test_ScVgIM0.zip')
# !unzip test_ScVgIM0.zip

In [107]:
test_df = pd.read_csv('test.csv')

In [92]:
test.shape

(10000, 1)

In [93]:
test

,id
0,60001
1,60002
2,60003
3,60004
4,60005
...,...
9995,69996
9996,69997
9997,69998
9998,69999


In [102]:
test_images = []
for i in tqdm(range(test_df.shape[0])):
    img = Image.open('test/'+test_df['id'][i].astype('str')+'.png')
    img = image.img_to_array(img)
    img = img/255
    test_images.append(img)
test = np.array(test_images)

100%|██████████| 10000/10000 [00:03<00:00, 3041.86it/s]


In [112]:
predictions = model.predict_classes(test)

In [113]:
predictions

array([9, 2, 1, ..., 8, 1, 5])

In [114]:
test_df['label'] = predictions
test_df

,id,label
0,60001,9
1,60002,2
2,60003,1
3,60004,1
4,60005,6
...,...,...
9995,69996,9
9996,69997,1
9997,69998,8
9998,69999,1


In [115]:
test_df.to_csv('submission_cnn.csv', header=True, index=False)